In [1]:
import numpy as np

In [2]:
X1 = np.load('X1.npy')
X2 = np.load('X2.npy')

X1.shape, X2.shape

((20, 15), (20, 15))

### Part 1

Compute empirical covariance matrices for both timeseries. Do they satisfy Eq. (2)?
$$
\Sigma = CC^T + \alpha \hat 1 \tag{2}
$$

In [3]:
# Наверное, в этом пункте нужно использовать np.cov / np.ma.cov
naive_cov_X1 = np.cov(X1.T)
naive_cov_X1.shape

(15, 15)

In [4]:
X2_masked = np.ma.masked_invalid(X2)
naive_cov_X2 = np.ma.cov(X2_masked.T)
naive_cov_X2

masked_array(
  data=[[2.4505264175212216, -0.3384539512233684, 1.197095342036254,
         -0.6575147210438123, 0.4988174017116548, 0.7389079322354934,
         0.478072541405609, 0.5114094912231902, -0.04181648647602972,
         -0.8293317887364263, -0.014680617592599558, 0.1470339240128133,
         0.2547666184508943, -0.09068333363550443, 0.3576595474895656],
        [-0.3384539512233684, 3.5538993875800395, -0.3547076774622889,
         1.610531590573879, -0.21356046040406526, 0.42651383377609714,
         0.20805823991728375, 0.376417510794167, -0.2220133938184207,
         -1.8045208945436, 0.5556168048189818, -0.2716729774963129,
         0.8384096658265215, 1.9061746420231231, -0.05405000438008067],
        [1.197095342036254, -0.3547076774622889, 1.708070563265975,
         -0.28454299484310414, 0.1864341862643171, -0.2049976723860083,
         -0.2792337773927089, -0.2941449093337135, 0.26154272211966406,
         -0.46883990753986443, -0.1285248973954149, -0.1136230035136

Чтобы определить, соответствуют ли матрицы уравнению (2), посчитаем их собственные числа. Если формула видна, то несколько собственных чисел должны быть одинаковыми и равными $\alpha$.

In [5]:
np.sort(np.linalg.eigvals(naive_cov_X1))
# Нет повторяющихся собственных значений, матрица не соответствует форумуле из задания

array([0.0452363 , 0.1262659 , 0.25504862, 0.52183941, 0.79272858,
       0.91984856, 1.25646141, 1.44782525, 2.09109087, 2.67577594,
       3.10339724, 3.70359375, 4.9544137 , 5.54719841, 7.8251243 ])

In [6]:
np.sort(np.linalg.eigvals(naive_cov_X2))
# Матрица не соответствует форумуле из задания и, более того, не является положительно определенной

array([-0.58751119, -0.34754566, -0.10019103,  0.43186791,  0.50523143,
        0.74795894,  1.19611302,  1.57591731,  1.70585949,  2.67782998,
        4.04580859,  4.29361651,  5.5859645 ,  7.63442631,  8.85052008])

In [7]:
# Проверим на всякий случай, что матрица хотя бы симметричная
(naive_cov_X2 == naive_cov_X2.T).all()

True

### Part 2

Show that if there is no missing data, $LL$ is a function of empirical covariance matrix $\Sigma_\text{emp}$ and of $\Sigma$.

Доказательство:

Правдоподобие является функцией от данных $X$ и от оцениваемого параметра $\Sigma$. Докажем, что от данных оно зависит только через $\Sigma_\text{emp}$.

$$
LL(\Sigma, X) = \frac{1}{T}\sum_{t=1}^T \ln P(\vec x_t | \Sigma)
$$
Для многомерного нормального распределения с нулевым средним
$$
\ln P(\vec x | \Sigma) =
- \frac12 x^T \Sigma^{-1} x 
- \frac12 \ln \det(\Sigma)
- \frac{n}{2} \ln(2\pi)
$$

Последние два слагаемых не зависят от $X$, поэтому
$$
LL(\Sigma, X) = - \frac1{2T} \underbrace{\sum_{t=1}^T x_t^T \Sigma^{-1} x_t}_{S} + g(\Sigma)
$$

Осталось вычислить сумму
$$
S = X_{ti} (\Sigma^{-1})_{ij} X_{tj} = \operatorname{Tr}(X^T X \Sigma^{-1})
$$
Вспомним, что при нулевых средних (смещённая) оценка ковариационной матрицы методом максимального правдоподобия равна
$$
\Sigma_\text{emp} = \frac1T X^T X
$$

Таким образом,
$$
LL(\Sigma, X) = 
- \frac12 \operatorname{Tr}(\Sigma_\text{emp} \Sigma^{-1})
- \frac12 \ln \det(\Sigma)
- \frac{n}{2} \ln(2\pi)
= f(\Sigma, \Sigma_\text{emp})
$$

### Part 3

For the timeseries-1, you thus have to maximize
$$
LL(C C^T + \alpha \hat 1, \sigma_\text{emp})
\tag{8}
$$
over $C$ and $\alpha > 0$. Set up the cost function in `JAX` and use some 2nd-order optimization algorithm.

Don’t forget to compile your cost function as well as its gradients for speed-up. For starting point of the
optimization algorithm, use some heuristic decomposition of $\Sigma_\text{emp}$ to the form of Eq. (2).

In [8]:
import jax.numpy as jnp
from jax import grad, jit
from jax.scipy.optimize import minimize

In [9]:
def negative_doubled_LL(Sigma, Sigma_empirical):
    return jnp.trace(Sigma_empirical @ jnp.linalg.inv(Sigma)) + jnp.linalg.slogdet(Sigma)[1]

In [10]:
def initial_guess(Sigma_empirical, rank: int):
    """Heuristic decomposition of covariance matrix
    in a form Sigma = C @ C.T + alpha * I
    using SVD decomposition"""

    U, S, VT = jnp.linalg.svd(Sigma_empirical)
    alpha = S[rank:].mean()
    C = U[:, :rank] * jnp.sqrt(S[:rank] - alpha)
    return jnp.hstack((alpha, C.flatten()))

In [11]:
def find_optimal_covariance(Sigma_empirical, rank: int, method='BFGS'):
    """Finds maximum-likelehood covariance matrix
    for multi-dimensional normal distribution
    in the form Sigma = C @ C.T + alpha * I
    where C is a matrix of shape (n_variables, rank) and alpha is a scalar."""

    n_variables = Sigma_empirical.shape[0]

    @jit
    def target_function(params):
        """Wrapper for negative_doubled_LL function
        that takes a vector of parameters
        [alpha, C_11, C_12, ..., C_nr]
        and returns negative doubled log-likelihood"""
        alpha = params[0]
        C = params[1:].reshape((n_variables, rank))
        Sigma = C @ C.T + alpha * jnp.eye(n_variables)
        return negative_doubled_LL(Sigma, Sigma_empirical)

    optimiz_result = minimize(target_function, initial_guess(Sigma_empirical, rank), method = method)

    alpha = optimiz_result.x[0]
    C = optimiz_result.x[1:].reshape((n_variables, rank))
    Sigma = C @ C.T + alpha * jnp.eye(n_variables)

    return Sigma, optimiz_result

In [12]:
regularized_cov_X1, optimiz_result = find_optimal_covariance(naive_cov_X1, 2)

regularized_cov_X1

Array([[ 1.9448285 ,  0.00573565,  0.13902299,  0.6531626 , -0.43983006,
        -0.25791845,  0.15218566,  0.01295092,  0.09997412,  0.23420574,
         0.2447613 ,  0.14591938, -0.38032365,  0.23381297,  0.50448745],
       [ 0.00573565,  1.9799254 ,  0.18269144,  0.45970252, -0.33617106,
        -0.12866504,  0.16885582,  0.29997277, -0.40292642, -0.5935469 ,
         0.12161588,  0.33235413, -0.05351539, -0.39962494, -0.06802653],
       [ 0.13902299,  0.18269144,  1.8673818 ,  0.61884546, -0.4328901 ,
        -0.2122535 ,  0.18170185,  0.1889728 , -0.19281185, -0.2388335 ,
         0.20113084,  0.27777284, -0.23023453, -0.12122595,  0.2209455 ],
       [ 0.6531626 ,  0.45970252,  0.61884546,  3.9912207 , -1.5936537 ,
        -0.83140594,  0.6305478 ,  0.48381284, -0.35970277, -0.31495607,
         0.7882622 ,  0.861305  , -1.020829  , -0.02385674,  1.144727  ],
       [-0.43983006, -0.33617106, -0.4328901 , -1.5936537 ,  2.786648  ,
         0.5709269 , -0.43949887, -0.35276714, 

In [13]:
# По какой-то причине nfev = 1,
# т.е. было 0 итераций и 
# ответ совпадает с начальным приближением

optimiz_result.nfev

Array(1, dtype=int32, weak_type=True)

### Part 4

To better understand the approach to the case of missing data, consider analytically the case of
one-step timeseries with single vector observation $x = (x_1,\ \text{nan},\ x_3)$ and proposed covariance matrix
$$
\Sigma = \begin{pmatrix}
1 & \rho & \rho \\
\rho & 1 & \rho \\
\rho & \rho & 1
\end{pmatrix}
$$
Integrate out unobserved value – compute $\int P(x_1, x_2, x_3) dx_2$ analytically.

Решение

Если удалить из многомерного нормального распределения одну переменную, то оно останется нормальным. У оставшихся переменных сохранится среднее и ковариация.

Поэтому при вычислении $\int P(x_1, x_2, x_3) dx_2$  мы должны будем получить нормальное распределение с обрезанной матрицей ковариации:
$$
P(\vec x') = 
(2\pi)^{-(n-1)/2} \det(\Sigma')^{-1/2}
\exp\biggl(
    -\frac12 x'^T \Sigma'^{-1} x'
\biggr),
$$
где $x' = [x_1, x_3],$

$
\Sigma' = \begin{pmatrix}
\operatorname{cov}(x_1, x_1) & \operatorname{cov}(x_1, x_3) \\
\operatorname{cov}(x_3, x_1) & \operatorname{cov}(x_3, x_3) 
\end{pmatrix}
= \begin{pmatrix}
1 & \rho \\
\rho & 1
\end{pmatrix}
$

Этот факт можно доказать, используя следующее определение многомерной нормальной случайной величины: $x$ — нормальная случайная величина, если найдётся такая матрица $A$ и вектор $\mu$, что
$$x = A\xi + \mu,$$ 
где $\xi$ — вектор из независимых стандартных нормальных случайных величин, $\mathcal{N}(0, 1)$. 

В таком случае $\Sigma = A^T A$.

Для удаления переменной мы удаляем соответствующую строку из матрицы $A$ и соответствующий элемент из вектора $\mu$. В результате из $\Sigma$ удаляется строка и столбец, соответствующие удалённой переменной, а остальные элементы остаются неизменными; распределение остаётся нормальным.

Так мы получили ответ, не выполняя интегрирование непосредственно.

### Part 5

Generalize the previous approach for arbitrary pattern of the missing data. Set up the corresponding
cost function for the timeseries-2 and optimize it over Σ, similarly to the case of the timeseries-1.

Пройдёмся по всем наблюдениям и посчитаем правдоподобие для каждого из них, используя полученную выше формулу с урезанной матрицей $\Sigma'$.

К сожалению, `JAX` не умеет работать с матрицами переменной размерности, поэтому придётся обойтись без автодифференциирования и использовать `numpy`.

Что-то находится.

In [44]:
from scipy.stats import multivariate_normal

In [45]:
def negative_doubled_LL_for_single_measurement_with_nans(Sigma, x):
    nan_positions = np.isnan(x)
    x = x[~nan_positions]
    Sigma = Sigma[~nan_positions, :][:, ~nan_positions]
    return multivariate_normal.logpdf(x, np.zeros_like(x), Sigma)

In [46]:
from scipy.optimize import minimize as minimize_scipy

In [52]:
def find_optimal_covariance_with_missing_values(X, rank: int, method='cobyla'):

    n_variables = X.shape[1]

    def target_function(params):
        """Wrapper for negative_doubled_LL function
        that takes a vector of parameters
        [alpha, C_11, C_12, ..., C_nr]
        and returns negative doubled log-likelihood"""
        alpha = params[0]
        C = params[1:].reshape((n_variables, rank))
        Sigma = C @ C.T + alpha * np.eye(n_variables)

        return sum(negative_doubled_LL_for_single_measurement_with_nans(Sigma, x) for x in X)


    X_masked = np.ma.masked_invalid(X)
    Sigma_empirical = np.ma.cov(X_masked.T)

    optimiz_result = minimize_scipy(target_function, initial_guess(Sigma_empirical, rank), method = method)

    alpha = optimiz_result.x[0]
    C = optimiz_result.x[1:].reshape((n_variables, rank))
    Sigma = C @ C.T + alpha * np.eye(n_variables)

    return Sigma, optimiz_result

In [61]:
regularized_cov_X2, optimiz_result = find_optimal_covariance_with_missing_values(X2, 2, 'cobyla')

regularized_cov_X2

array([[ 9910.37815947,  4714.60466397,  6641.57752699,  7614.39510964,
         6503.83885158,  6389.40762871,  4815.15934583, -2089.21728016,
         6269.15450435,  5376.34154976,  6934.4593213 ,  1555.23623094,
        -3487.50449662, -2708.31445264,  6871.36569543],
       [ 4714.60466397,  3846.64720538,  2717.31021151,  4185.32955208,
         2604.85314644,  3486.85794918,  2778.59740678, -1460.1976194 ,
         3899.67062738,  1477.90647071,  3793.35284744,  3545.81003051,
         2717.91213605,  2144.54892719,  5974.32589379],
       [ 6641.57752699,  2717.31021151,  5643.11595874,  5156.95392013,
         5042.4701325 ,  4343.84308257,  3174.43170563, -1209.96605483,
         3947.59042622,  4612.26640443,  4708.45004796,  -715.58195461,
        -5408.6568725 , -4222.50855939,  3209.27145015],
       [ 7614.39510964,  4185.32955208,  5156.95392013,  6848.76453362,
         5028.77279179,  5291.42712244,  4044.7902134 , -1851.32248411,
         5372.90004693,  3901.4012449

In [62]:
optimiz_result

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -1086.93047072318
       x: [ 5.315e+02  3.279e+01 ...  1.012e+02  3.901e+01]
    nfev: 1000
   maxcv: 0.0